In [1]:
import pickle
import datetime as dt
import os
#import time
#import matplotlib.pyplot as plt
#%matplotlib inline
#import dateutil.parser as dp
#import scipy.stats as stat
#import numpy as np
#import pandas as pd

Finding the repository we are working in so later is easier to read/write files

In [2]:
dir_path = os.path.abspath('')
data_path = os.path.join(dir_path, 'data')
print(data_path)
#path_for_saving_estimations = "/m/cs/scratch/networks/heydars1/mobility-visualizations-finland/hcd_level_od_estimaion_of_different_models/"
path_for_saving_estimations = os.path.join(dir_path, 'results')

/scratch/cs/networks/heydars1/dynamic-mobility/data


In [3]:
def date_to_weekday(day, month, year):
    parsed_date = dt.date(int(year), int(month), int(day))
    weekday_map = {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
    return((weekday_map[parsed_date.weekday()]))

# Road Traffic

In [4]:
def calculate_total_out_in_traffic(traffic_between_adjacent_data_path, out_degree_save_path = None, in_degree_save_path = None):
    ##header of thte data and a example line:
    #origin, destination, day, month, year, hour, vehicle-type:1, vehicle-type:2, vehicle-type:3, vehicle-type:4, vehicle-type:5, vehicle-type:6, vehicle-type:7
    #Central-Finland-Hospital-District,Central-Ostrobothnia-Hospital-District,1,2,2020,0,12.0,0.0,0.0,0.0,11.0,0.0,0.0
    hcd_in_degree = {} # a dictionary with (destination, hour, day, month, year) as key and total in-degree as value
    hcd_out_degree = {}
    with open(traffic_between_adjacent_data_path, 'r') as data:
        for line in data:
            fields = line.strip().split(",")
            origin = fields[0]
            destination = fields[1]
            day, month, year, hour = int(fields[2]), int(fields[3]), int(fields[4]), int(fields[5])
            vehicle_count_list = [float(fields[6]), float(fields[7]), float(fields[8]), float(fields[9]), float(fields[10]), float(fields[11]), float(fields[12])]
            if (destination, hour, day, month, year) not in hcd_in_degree.keys():
                hcd_in_degree[(destination, hour, day, month, year)] = [0 for i in range(len(vehicle_count_list))]
            updated_in = [a + b for a, b in zip(hcd_in_degree[(destination, hour, day, month, year)], vehicle_count_list)]
            hcd_in_degree[(destination, hour, day, month, year)] = updated_in
            if (origin, hour, day, month, year) not in hcd_out_degree.keys():
                hcd_out_degree[(origin, hour, day, month, year)] = [0 for i in range(len(vehicle_count_list))]
            updated_out = [a + b for a, b in zip(hcd_out_degree[(origin, hour, day, month, year)], vehicle_count_list)]
            hcd_out_degree[(origin, hour, day, month, year)] = updated_out
            
    ### print the in degree and out degree to a file
    #print(out_degree_save_path)
    if out_degree_save_path is not None:
        #print("inside out-degree saving if")
        out_key_list = list(hcd_out_degree.keys())
        sorted_out_key_list = sorted(out_key_list,key=lambda x: (x[0], x[4], x[3], x[2], x[1]))
        with open(out_degree_save_path, "w+") as out_save_file:
            header = "in-or-out, hcd, hour, day, month, year, type-1-count, type-2-count, type-3-count, type-4-count, type-5-count, type-6-count, type-7-count"
            out_save_file.write(header+"\n")
            for key in sorted_out_key_list:
                out_save_file.write("outdegree,"+','.join([str(key[0]),str(key[1]),str(key[2]),str(key[3]),str(key[4])])+','+','.join([str(i) for i in hcd_out_degree[key]])+"\n")
                
                
    if in_degree_save_path is not None:
        in_key_list = list(hcd_in_degree.keys())
        sorted_in_key_list = sorted(in_key_list,key=lambda x: (x[0], x[3], x[2], x[1]))
        output_file = open(in_degree_save_path, "w+")
        header = "in-or-out, hcd, hour, day, month, year, type-1-count, type-2-count, type-3-count, type-4-count, type-5-count, type-6-count, type-7-count"
        output_file.write(header+"\n")
        for key in sorted_in_key_list:
            output_file.write("indegree,"+','.join([str(key[0]),str(key[1]),str(key[2]),str(key[3]),str(key[4])])+','+ ','.join([str(i) for i in hcd_in_degree[key]])+"\n")
        
    return(hcd_in_degree, hcd_out_degree)    

In the next cell we determine the path to road traffic data .csv file which is aggregated in hospital district care level

In [5]:
road_traffic_datapath = dir_path+ "/road_traffic_data/hcd_2019_and_2020_4months_each_hcd_level_6h.csv"
print(road_traffic_datapath)

/scratch/cs/networks/heydars1/dynamic-mobility/road_traffic_data/hcd_2019_and_2020_4months_each_hcd_level_6h.csv


In [6]:
#path_to_save_hcd_road_indegree = os.path.join(dir_path, 'total_hcd_road_traffic_indegree_outdegree/hcd_road_indegree.csv')
#path_to_save_hcd_road_outdegree = os.path.join(dir_path, 'total_hcd_road_traffic_indegree_outdegree/hcd_road_outdegree.csv')
#print(path_to_save_hcd_road_indegree)
#print(path_to_save_hcd_road_outdegree)
#hcd_traffic_in_degree, hcd_traffic_out_degree = calculate_total_out_in_traffic(road_traffic_datapath, path_to_save_hcd_road_outdegree, path_to_save_hcd_road_indegree)

In [7]:
hcd_traffic_in_degree, hcd_traffic_out_degree = calculate_total_out_in_traffic(road_traffic_datapath)

the following function combines temporal road traffic data with the fractions from static radiation model to make a dynamic radiation model

In [8]:
def combine_road_traffic_with_static_fractions(in_road_traffic_count_dictionary, in_fractions_dictionary, out_road_traffic_count_dictionary, out_fractions_dictionary, passenger_per_car, path_to_save_results = None):
    regions_in_road_traffic = set([tup[0] for tup in in_road_traffic_count_dictionary.keys()])
    regions_in_static = set([tup[0] for tup in in_fractions_dictionary.keys()])
    print(regions_in_road_traffic)
    print(regions_in_static)
    set_of_regions_in_both_data_sources = regions_in_road_traffic.intersection(regions_in_static)
    region_list = list(set_of_regions_in_both_data_sources)
    print(len(set_of_regions_in_both_data_sources))
    prediction_from_in_flow = {}
    prediction_from_out_flow = {}
    
    for key in in_road_traffic_count_dictionary.keys():
        destination = key[0]
        if destination in set_of_regions_in_both_data_sources:
            vehicle_count_sum = sum(in_road_traffic_count_dictionary[key])
            passenger_sum = vehicle_count_sum * passenger_per_car
            hour = str(key[1])
            day = str(key[2])
            month = str(key[3])
            year = str(key[4])
            if hour in {"0", "6"}: #morning
                for origin in region_list:
                    if origin != destination:
                        frac = in_fractions_dictionary[origin, destination]
                        result = passenger_sum * frac
                        prediction_from_in_flow[(origin, destination, hour, day, month, year)] = result
                        

            elif hour in {"12", "18"}: #afternoon
                for origin in region_list:
                    if origin != destination:
                        frac = out_fractions_dictionary[origin, destination]
                        result = passenger_sum * frac
                        prediction_from_in_flow[(origin, destination, hour, day, month, year)] = result
            #from IPython.core.debugger import Pdb
            #Pdb().set_trace()

            
    
    for key in out_road_traffic_count_dictionary.keys(): 
        origin = key[0]
        if origin in set_of_regions_in_both_data_sources:
            vehicle_count_sum = sum(out_road_traffic_count_dictionary[key])
            passenger_sum = vehicle_count_sum * passenger_per_car
            hour = str(key[1])
            day = str(key[2])
            month = str(key[3])
            year = str(key[4])
            if hour in {"0", "6"}: #morning
                for destination in region_list:
                    if origin != destination:
                        frac = out_fractions_dictionary[origin, destination]
                        result = passenger_sum * frac
                        prediction_from_out_flow[(origin, destination, hour, day, month, year)] = result
            elif hour in {"12", "18"}: #afternoon
                for destination in region_list:
                    if origin != destination:
                        frac = in_fractions_dictionary[origin, destination]
                        result = passenger_sum * frac
                        prediction_from_out_flow[(origin, destination, hour, day, month, year)] = result
    
    
    mutual_in_and_out_prediction_key_set = set(prediction_from_out_flow.keys()).intersection(set(prediction_from_in_flow))
    print("mutual keys")
    print(len(mutual_in_and_out_prediction_key_set))
    combined_prediction = {}
    for key in prediction_from_in_flow.keys():
        combined_prediction[key] = (prediction_from_in_flow[key] + prediction_from_out_flow[key])/2.0
    if path_to_save_results:
        print("printing")
        #write the files in a simillar formatting to the operator data
        with open(path_to_save_results, 'w') as results_file:
            header_list = ["flow_estimate", "timestamp", "date", "day_of_the_week", "hour", "origin_hcd", "destination_hcd"]
            header_to_write = ",".join(header_list)+"\n"
            results_file.write(header_to_write)
            for key in combined_prediction.keys():
                hour = str(key[2])
                day = str(key[3])
                month = str(key[4])
                year = str(key[5])
                #date_string = str(key[5])+"-"+str(key[4])+"-"+str(key[3])
                date_string = year+"-"+month+"-"+day
                weekday = date_to_weekday(day, month, year)
                datetime_string = year+"-"+month+"-"+day+" "+hour+":00"
                list_to_write = [str(combined_prediction[key]), datetime_string, date_string, weekday, hour, key[0], key[1]]
                line_to_write = ",".join(list_to_write)+"\n"
                results_file.write(line_to_write)         
    return combined_prediction

# Radiation Model
read the estimates of static radiation model in the level of hospital care districts from the saved pickle file

In [9]:
radiation_static_commuters_book = {}
hospital_radiation_path = (path_for_saving_estimations+"/radiation_model_commuters_hcd.pickle")
with open(hospital_radiation_path,'rb') as f:
     radiation_static_commuters_book = pickle.load(f)

The following function calculates the outgoing and incomming fractions for orign-destination pairs when the commute inside regions is excluded (since population units when calculating radiation model results are municipalities a large fraction of predicted commute is inside hospital care districts).

In [10]:
def calculate_excluded_fractions_from_static_model_results(static_results_book):
    #the fractions excluding inside-region mobility
    incomming_frac_book = {}
    outgoing_frac_book = {}
    sum_incomming = {}
    sum_outgoing = {}
    for (origin, destination) in static_results_book.keys():
        if origin != destination:
            value = static_results_book[(origin, destination)]
            if origin not in sum_outgoing.keys():
                sum_outgoing[origin] = 0
            sum_outgoing[origin] += value
            if destination not in sum_incomming.keys():
                sum_incomming[destination] = 0
            sum_incomming[destination] += value
    for (origin, destination) in static_results_book.keys():
        if origin != destination:
            outgoing_frac_book[(origin, destination)] = static_results_book[(origin, destination)]/sum_outgoing[origin]
            incomming_frac_book[(origin, destination)] = static_results_book[(origin, destination)]/sum_incomming[destination]
    return(outgoing_frac_book,incomming_frac_book)
            
        

In [11]:
radiation_out_excluding_frac, radiation_in_excluding_frac = calculate_excluded_fractions_from_static_model_results(radiation_static_commuters_book)

# Gravity Models
Gravity model with commuters' data from statistics Finland

In [12]:
# Load data (deserialize)
gravity_static_commuters_path = path_for_saving_estimations + "/gravity_model_commuters_hcd.pickle"
gravity_static_commuters_book = {}
with open(gravity_static_commuters_path, 'rb') as handle:
    gravity_static_commuters_book = pickle.load(handle)
    

In [13]:
len(gravity_static_commuters_book)

150

In [14]:
# the estimations of one the gravity models is missing some of the od values (wher the model predicts near zero values)
for key in radiation_static_commuters_book.keys():
    if key not in gravity_static_commuters_book.keys():
        gravity_static_commuters_book[key] = 0

In [15]:
len(gravity_static_commuters_book)

441

## Combine road data with gravity model to get dynamic estimations

In [16]:
gravity_commuters_out_excluding_frac, gravity_commuters_in_excluding_frac = calculate_excluded_fractions_from_static_model_results(gravity_static_commuters_book)

In [17]:
passengers_per_car = 1
dynamic_commuters_gravity_estimation_book = combine_road_traffic_with_static_fractions(hcd_traffic_in_degree, gravity_commuters_in_excluding_frac, hcd_traffic_out_degree, gravity_commuters_out_excluding_frac, passengers_per_car, None)

{'Central Ostrobothnia Hospital District', 'North Karelia Hospital District', 'Vaasa Hospital District', 'Central Finland Hospital District', 'Länsi-Pohja Hospital District', 'Satakunta Hospital District', 'North Savo Hospital District', 'Pirkanmaa Hospital District', 'Kymenlaakso Hospital District', 'Helsinki and Uusimaa Hospital District', 'Kanta-Häme Hospital District', 'North Ostrobothnia Hospital District', 'Kainuu Hospital District', 'South Karelia Hospital District', 'South Ostrobothnia Hospital District', 'Itä-Savo Hospital District', 'Lappi Hospital District', 'South Savo Hospital District', 'Southwest Finland Hospital District', 'Päijät-Häme Hospital District'}
{'Central Ostrobothnia Hospital District', 'North Karelia Hospital District', 'Vaasa Hospital District', 'Åland', 'Central Finland Hospital District', 'Länsi-Pohja Hospital District', 'Satakunta Hospital District', 'North Savo Hospital District', 'Pirkanmaa Hospital District', 'Kymenlaakso Hospital District', 'Helsinki

In [18]:
# pickle estimations
save_path = path_for_saving_estimations + "/dynamic_road_commuters_gravity_book.pkl"
with open(save_path, 'wb') as handle:
    pickle.dump(dynamic_commuters_gravity_estimation_book, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Combine road data with radiation model to get dynamic estimations

In [19]:
passengers_per_car = 1
#path_to_save_results = os.path.join(dir_path, 'intermediate_results/estimate_from_radiation_and_road_traffic_another_format.csv')
#print(path_to_save_results)
#radiation_combined_estimation_book = combine_road_traffic_with_static_fractions(hcd_traffic_in_degree, radiation_in_excluding_frac, hcd_traffic_out_degree, radiation_out_excluding_frac, passengers_per_car, path_to_save_results)
dynamic_commuters_radiation_estimation_book = combine_road_traffic_with_static_fractions(hcd_traffic_in_degree, radiation_in_excluding_frac, hcd_traffic_out_degree, radiation_out_excluding_frac, passengers_per_car, None)

{'Central Ostrobothnia Hospital District', 'North Karelia Hospital District', 'Vaasa Hospital District', 'Central Finland Hospital District', 'Länsi-Pohja Hospital District', 'Satakunta Hospital District', 'North Savo Hospital District', 'Pirkanmaa Hospital District', 'Kymenlaakso Hospital District', 'Helsinki and Uusimaa Hospital District', 'Kanta-Häme Hospital District', 'North Ostrobothnia Hospital District', 'Kainuu Hospital District', 'South Karelia Hospital District', 'South Ostrobothnia Hospital District', 'Itä-Savo Hospital District', 'Lappi Hospital District', 'South Savo Hospital District', 'Southwest Finland Hospital District', 'Päijät-Häme Hospital District'}
{'Central Ostrobothnia Hospital District', 'North Karelia Hospital District', 'Vaasa Hospital District', 'Åland', 'Central Finland Hospital District', 'Länsi-Pohja Hospital District', 'Satakunta Hospital District', 'North Savo Hospital District', 'Pirkanmaa Hospital District', 'Kymenlaakso Hospital District', 'Helsinki

In [20]:
# pickle estimations
save_path = path_for_saving_estimations + "/radiation_road_estimations.pkl"
with open(save_path, 'wb') as handle:
    pickle.dump(dynamic_commuters_radiation_estimation_book, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
dynamic_commuters_radiation_estimation_book

{('North Karelia Hospital District',
  'Central Ostrobothnia Hospital District',
  '0',
  '1',
  '2',
  '2019'): 1.1382074498176085,
 ('Vaasa Hospital District',
  'Central Ostrobothnia Hospital District',
  '0',
  '1',
  '2',
  '2019'): 154.13676202182393,
 ('Central Finland Hospital District',
  'Central Ostrobothnia Hospital District',
  '0',
  '1',
  '2',
  '2019'): 36.0409315832611,
 ('Länsi-Pohja Hospital District',
  'Central Ostrobothnia Hospital District',
  '0',
  '1',
  '2',
  '2019'): 3.9876500301215954,
 ('Satakunta Hospital District',
  'Central Ostrobothnia Hospital District',
  '0',
  '1',
  '2',
  '2019'): 0.5025944288150692,
 ('North Savo Hospital District',
  'Central Ostrobothnia Hospital District',
  '0',
  '1',
  '2',
  '2019'): 8.58471092568537,
 ('Pirkanmaa Hospital District',
  'Central Ostrobothnia Hospital District',
  '0',
  '1',
  '2',
  '2019'): 2.704923684408211,
 ('Kymenlaakso Hospital District',
  'Central Ostrobothnia Hospital District',
  '0',
  '1',
